## 1. 模型简介
PP-YOLOE是基于PP-YOLOv2的卓越的单阶段Anchor-free模型，超越了多种流行的YOLO模型。PP-YOLOE有一系列的模型，即s/m/l/x，可以通过width multiplier和depth multiplier配置。PP-YOLOE避免了使用诸如Deformable Convolution或者Matrix NMS之类的特殊算子，以使其能轻松地部署在多种多样的硬件上。关于PP-YOLOE的更多细节可以参考我们的[官方文档](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/configs/ppyoloe/README_cn.md)。

## 2. 模型效果
PP-YOLOE-l在COCO test-dev2017达到了51.6的mAP, 同时其速度在Tesla V100上达到了78.1 FPS。如下图所示，PP-YOLOE-s/m/x同样具有卓越的精度速度性价比。
<div align="center">
  <img src="https://paddledet.bj.bcebos.com/modelcenter/images/ppyoloe_map_fps.png" width=500 />
</div>

## 3. 模型如何使用
首先克隆PaddleDetection，并将数据集存放在`dataset/coco/`目录下面

In [ ]:
%cd ~/work
!git clone https://gitee.com/paddlepaddle/PaddleDetection
%cd PaddleDetection

### 3.1 训练
执行以下指令训练PP-YOLOE

In [ ]:
# 单卡训练
!python tools/train.py -c configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml --eval --amp

# 多卡训练
!python -m paddle.distributed.launch --gpus 0,1,2,3,4,5,6,7 tools/train.py -c configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml --eval --amp

**注意：**
- 如果需要边训练边评估，请添加`--eval`.
- PP-YOLOE支持混合精度训练，请添加`--amp`.

### 3.2 推理部署
#### 3.2.1 使用Paddle-TRT进行推理部署
接下来，我们将介绍PP-YOLOE如何使用Paddle Inference在TensorRT FP16模式下部署

首先，参考[Paddle Inference文档](https://www.paddlepaddle.org.cn/inference/master/user_guides/download_lib.html#python)，下载并安装与你的CUDA, CUDNN和TensorRT相应的wheel包。

然后，运行以下命令导出模型

In [ ]:
!python tools/export_model.py -c configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml -o weights=https://paddledet.bj.bcebos.com/models/ppyoloe_crn_l_300e_coco.pdparams trt=True

最后，使用TensorRT FP16进行推理

In [ ]:
# 推理单张图片
!CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/ppyoloe_crn_l_300e_coco --image_file=demo/000000014439_640x640.jpg --device=gpu --run_mode=trt_fp16

# 推理文件夹下的所有图片
!CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/ppyoloe_crn_l_300e_coco --image_dir=demo/ --device=gpu  --run_mode=trt_fp16

**注意：**
- TensorRT会根据网络的定义，执行针对当前硬件平台的优化，生成推理引擎并序列化为文件。该推理引擎只适用于当前软硬件平台。如果你的软硬件平台没有发生变化，你可以设置[enable_tensorrt_engine](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/deploy/python/infer.py#L857)的参数`use_static=True`，这样生成的序列化文件将会保存在`output_inference`文件夹下，下次执行TensorRT时将加载保存的序列化文件。
- PaddleDetection release/2.4及其之后的版本将支持NMS调用TensorRT，需要依赖PaddlePaddle release/2.3及其之后的版本。

#### 3.2.2 使用PaddleInference进行推理部署
对于一些不支持TensorRT的AI加速硬件，我们可以直接使用PaddleInference进行部署。首先运行以下命令导出模型

In [ ]:
!python tools/export_model.py -c configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml -o weights=https://paddledet.bj.bcebos.com/models/ppyoloe_crn_l_300e_coco.pdparams

直接使用PaddleInference进行推理

In [ ]:
# 推理单张图片
!CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/ppyoloe_crn_l_300e_coco --image_file=demo/000000014439_640x640.jpg --device=gpu --run_mode=paddle

# 推理文件夹下的所有图片
!CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/ppyoloe_crn_l_300e_coco --image_dir=demo/ --device=gpu  --run_mode=paddle

## 4. 模型原理
PP-YOLOE的整体结构图如下所示：
<div align="center">
  <img src="https://paddledet.bj.bcebos.com/modelcenter/images/PP-YOLOE-Arch.png" width=70% />
</div>
PP-YOLOE由以下方法组成:

- 可扩展的backbone和neck
- [Task Alignment Learning](https://arxiv.org/abs/2108.07755)
- Efficient Task-aligned head with [DFL](https://arxiv.org/abs/2006.04388)和[VFL](https://arxiv.org/abs/2008.13367)
- [SiLU(Swish)激活函数](https://arxiv.org/abs/1710.05941)

更多细节可以参考我们的技术报告：https://arxiv.org/abs/2203.16250

## 5. 注意事项
**所有的命令默认运行在AI Studio的`jupyter`上, 如果运行在终端上，去掉命令开头的符号%或!**

## 6. 相关论文及引用信息
```
@article{xu2022pp,
  title={PP-YOLOE: An evolved version of YOLO},
  author={Xu, Shangliang and Wang, Xinxin and Lv, Wenyu and Chang, Qinyao and Cui, Cheng and Deng, Kaipeng and Wang, Guanzhong and Dang, Qingqing and Wei, Shengyu and Du, Yuning and others},
  journal={arXiv preprint arXiv:2203.16250},
  year={2022}
}
```